## https://medium.com/pew-research-center-decoded/using-fixed-and-random-effects-models-for-panel-data-in-python-a795865736ab

and 
## https://bashtage.github.io/linearmodels/panel/examples/examples.html

Identifying causal relationships from observational data is not easy. Still, researchers are often interested in examining the effects of policy changes or other decisions. In those analyses, researchers will face any number of analytical decisions, including whether to use fixed or random effects models to control for variables that don’t change over time.


Let’s consider an example. Suppose we’re interested in estimating the effect that a government grant might have had on firms’ product quality (as examined in this previous study). In addition to controlling for observed variables like the number of employees the firms had at different time points in the study period, we might also want to control for unobserved variables, such as the management quality of the firms.


Assuming that the firms’ management quality is constant over time, we can use regression models to try to account for those unobserved factors — but there isn’t always consensus about the best way to do so. Specifically, researchers often must decide whether to use a fixed or random effects approach in an analysis like this.


Fixed vs. random effects in panel data
Broadly speaking, the distinction between a fixed effects approach and a random effects approach concerns the correlation — or lack thereof — between unobserved variables and observed variables. To highlight this difference, let’s go back to the example cited above.


The key issue in deciding between the two approaches is whether or not the unobserved variables in our analysis — in this case, the firms’ management quality — might be correlated with observed variables. We might use a fixed effects approach if we think that these variables are correlated — for example, if we think firms’ management quality has a role in determining whether the firms receive a grant. But we might use a random effects approach if we think the two variables are not correlated.


Fixed effects help capture the effects of all variables that don’t change over time. In other words, anything else that does not change over time at the firm level, such as its location, would be captured by these fixed effects terms in the model. That means we cannot separately estimate the effect of firms’ location on their performance.


This is quite restrictive for some applications, so researchers who might be interested in studying the effect of time-invariant variables may want to choose the random effects framework instead, even though these models impose stronger assumptions about the unobserved effects.

In [53]:

#pip install linearmodels in terminal or command line
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects


To implement a random effects model, we call the RandomEffects method and assign the firm code and year columns as the indexes in the dataframe.



In [69]:


# Load the CSV (load date data as proper date types)
df = pd.read_csv("data/tsdata_movingav_narm.csv")
#In our case: assign the label and the timestamp as the indices in the data frame
from datetime import datetime
df['date2'] = pd.to_datetime(df['date2'])
date2 = pd.Categorical(df.date2)
df = df.set_index(['Label', 'date2'])
df['date2'] = date2

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1200555 entries, ('Yahoo', Timestamp('2015-01-01 00:00:00')) to ('8world%20News', Timestamp('2019-12-30 00:00:00'))
Data columns (total 41 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   id                   1200555 non-null  object  
 1   sno                  1200555 non-null  float64 
 2   date                 1200555 non-null  object  
 3   subscriberCount      1200555 non-null  int64   
 4   topic_fin            1200555 non-null  object  
 5   like_03              1200555 non-null  float64 
 6   share_03             1200555 non-null  float64 
 7   comment_03           1200555 non-null  float64 
 8   love_03              1200555 non-null  float64 
 9   wow_03               1200555 non-null  float64 
 10  haha_03              1200555 non-null  float64 
 11  sad_03               1200555 non-null  float64 
 12  angry_03             1200555 non-null  float64 
 13  t

For the dependent variable, we use the change in scrap rate between periods as a proxy of the product quality. For the independent variables, we include the grant status in period t (=1 if received grant) and the number of employees at the firm.


In [63]:
# in our case, 7 day average of like, share, comment, love on a topic should predict whether the next article is about that
# and we run this for each outlet at a time
# binary transformation of the topic_fin variable
# 7 day moving average of like, share, comment, love

import statsmodels.api as sm
from linearmodels.panel import PooledOLS

exog_vars = ['log1p_likeCount', 'log1p_shareCount','log1p_commentCount','log1p_loveCount','log1p_wowCount','log1p_hahaCount','log1p_sadCount','log1p_angryCount','log1p_careCount']
exog = sm.add_constant(df[exog_vars])
mod = RandomEffects(df.topic_celebrity, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:        topic_celebrity   R-squared:                        0.0002
Estimator:              RandomEffects   R-squared (Between):              0.0286
No. Observations:             1200555   R-squared (Within):               0.0002
Date:                Sun, Mar 06 2022   R-squared (Overall):              0.0002
Time:                        10:31:27   Log-likelihood                 1.203e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.738
Entities:                          33   P-value                           0.0000
Avg Obs:                    3.638e+04   Distribution:               F(9,1200545)
Min Obs:                       3.0000                                           
Max Obs:                    1.985e+05   F-statistic (robust):             24.738
                            

In [57]:
#Example data from the article

from linearmodels.datasets import jobtraining
data = jobtraining.load()
year = pd.Categorical(data.year)
data = data.set_index(['fcode', 'year'])
data['year'] = year
data

employ       sales   avgsal  scrap  rework  tothrs  union  grant  \
fcode  year                                                                     
410032 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0      0   
       1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0      0   
       1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0      0   
410440 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0      0   
       1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0      0   
...             ...         ...      ...    ...     ...     ...    ...    ...   
419483 1988   108.0  11500000.0  14810.0   25.0     NaN     0.0      1      0   
       1989   129.0  12000000.0  14227.0   30.0     NaN    20.0      1      0   
419486 1987    80.0   7000000.0  16000.0    NaN     NaN     0.0      0      0   
       1988    90.0   8500000.0  17000.0    NaN     NaN     0.0      0      0   
       1989   100.0   9900000.0  18000.0    NaN     NaN    40.0      0      1   

             d89  d88  ...   clscrap  cgrant  clemploy   clsales    lavgsal  \
fcode  year            ...                                                    
410032 1987    0    0  ...       NaN       0       NaN       NaN  10.463100   
       1988    0    1  ...       NaN       0  0.270027 -0.088949  10.518670   
       1989    1    0  ...       NaN       0 -0.063013  0.130621  10.571320   
410440 1987    0    0  ...       NaN       0       NaN       NaN   9.259130   
       1988    0    1  ...       NaN       0  0.080043  0.233347   9.305651   
...          ...  ...  ...       ...     ...       ...       ...        ...   
419483 1988    0    1  ...  0.223144       0 -0.208218  0.044453   9.603058   
       1989    1    0  ...  0.182321       0  0.177681  0.042559   9.562897   
419486 1987    0    0  ...       NaN       0       NaN       NaN   9.680344   
       1988    0    1  ...       NaN       0  0.117783  0.194157   9.740969   
       1989    1    0  ...       NaN       1  0.105360  0.152469   9.798127   

             clavgsal  cgrant_1    chrsemp  clhrsemp  year  
fcode  year                                                 
410032 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.055570       0.0  -8.946565 -1.165385  1988  
       1989  0.052644       0.0   0.198597  0.047832  1989  
410440 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.046520       0.0   0.000000  0.000000  1988  
...               ...       ...        ...       ...   ...  
419483 1988  0.059321       0.0   0.000000  0.000000  1988  
       1989 -0.040161       0.0   3.100775  1.411176  1989  
419486 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.060625       0.0   0.000000  0.000000  1988  
       1989  0.057159       0.0  36.000000  3.610918  1989  

[471 rows x 29 columns]

In [42]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

exog_vars = ['grant', 'employ']
exog = sm.add_constant(data[exog_vars])
mod = RandomEffects(data.clscrap, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                clscrap   R-squared:                        0.0165
Estimator:              RandomEffects   R-squared (Between):              0.0314
No. Observations:                 105   R-squared (Within):               0.0015
Date:                Sat, Mar 05 2022   R-squared (Overall):              0.0199
Time:                        22:53:09   Log-likelihood                   -77.721
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.8542
Entities:                          53   P-value                           0.4286
Avg Obs:                       1.9811   Distribution:                   F(2,102)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             0.8634
                            

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\shared\exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## PanelOLS

To implement the fixed effects model, we use the PanelOLS method, and set the parameter `entity_effects` to be True.

PanelOLS can be used to estimated models with up to 2-effects fixed effects. These can include any combination of

Entity effects

Time effects

Other effects


In [65]:
mod = PanelOLS(df.topic_celebrity, exog,entity_effects=True)
re_res = mod.fit()
print(re_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:        topic_celebrity   R-squared:                        0.0002
Estimator:                   PanelOLS   R-squared (Between):             -0.0090
No. Observations:             1200555   R-squared (Within):               0.0002
Date:                Sun, Mar 06 2022   R-squared (Overall):              0.0002
Time:                        10:32:39   Log-likelihood                 1.203e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.639
Entities:                          33   P-value                           0.0000
Avg Obs:                    3.638e+04   Distribution:               F(9,1200513)
Min Obs:                       3.0000                                           
Max Obs:                    1.985e+05   F-statistic (robust):             24.639
                            

The results are quite different between the fixed and random effects models, but neither is statistically significant. However, to the extent that you think the unobserved effect of the firms is uncorrelated with whether the firms received the grant, the random effects model is more appropriate.


Equivalence of fixed effects model and dummy variable regression

Estimating a fixed effects model is equivalent to adding a dummy variable for each subject or unit of interest in the standard OLS model. To illustrate equivalence between the two approaches, we can use the OLS method in the statsmodels library, and regress the same dependent variable on the categorized variable of firm, and other independent variables:

In [ ]:
data = jobtraining.load()
data[‘year’] = pd.Categorical(data.year)
FE_ols = smf.ols(formula=’clscrap ~ 1 + grant + employ + C(fcode)’, data = data).fit()
print(FE_ols.summary())

In [72]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
df = pd.read_csv("data/tsdata_movingav_narm.csv")
FE_ols = smf.ols(formula='topic_celebrity ~ 1 + log1p_likeCount + log1p_shareCount + log1p_commentCount + log1p_loveCount + log1p_wowCount + log1p_hahaCount + log1p_sadCount + log1p_angryCount + C(Label)', data = df).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:        topic_celebrity   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     15.99
Date:                Sun, 06 Mar 2022   Prob (F-statistic):          4.37e-109
Time:                        10:40:45   Log-Likelihood:             1.2030e+06
No. Observations:             1200555   AIC:                        -2.406e+06
Df Residuals:                 1200514   BIC:                        -2.405e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

The results from the dummy regression show the separately estimated effect of each firm on change in scrap rate. This is sometimes useful when we want to focus on specific units. In addition, we can compute some sample averages of these estimates to get a sense of how much variation there is across firms.

Time fixed effects

Time effect can be added using time_effects=True. Here the time dummies are removed. Note that the core coefficients are identical. The only change is in the test statistic for poolability since not the “effects” include both entity and time, whereas before only entity were included.

In [66]:
mod = PanelOLS(df.topic_celebrity, exog,entity_effects=True,time_effects = True)
fe_te_res = mod.fit()
print(fe_te_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:        topic_celebrity   R-squared:                        0.0002
Estimator:                   PanelOLS   R-squared (Between):             -0.0626
No. Observations:             1200555   R-squared (Within):               0.0002
Date:                Sun, Mar 06 2022   R-squared (Overall):              0.0002
Time:                        10:33:31   Log-likelihood                 1.204e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.622
Entities:                          33   P-value                           0.0000
Avg Obs:                    3.638e+04   Distribution:               F(9,1198693)
Min Obs:                       3.0000                                           
Max Obs:                    1.985e+05   F-statistic (robust):             27.622
                            